Olá Sou Janatã Aruanda, Estou Aprendendo e treinando o spark, com essa atividade Spark com ML, irei deixar comentando e explicado cada célula 


Montando o Ambiente no Colab

In [35]:
# instalar as dependências
# baixando a versão mais recente do java8;
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#download via stp do hadoop
!wget -q https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
#extraindo o arquivo tgz
!tar xf /content/spark-3.3.1-bin-hadoop3.tgz
#instalando o  Findspark que é responsavel para que as variáveis ​​do ambiente sejam definidas corretamente e o pyspark seja importado.
!pip install -q findspark

In [36]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

# utilizando o findspark para que o pyspark seja "importável"
import findspark
findspark.init('spark-3.3.1-bin-hadoop3')

In [37]:
#Importação da biblioteca ML
from pyspark.ml.regression import LinearRegression, RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator 
from pyspark.ml.feature import VectorAssembler

In [38]:
#Importações para começar a sessão 
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
spark = SparkSession.builder.getOrCreate()

Manipulação 


In [39]:
#carregando o arquivo 
Carros_temp = spark.read.csv("/content/base/Carros.csv", inferSchema=True, header=True, sep=";")

In [40]:
#imprimindo a tabela carregado anteriomente
Carros_temp.show(5)

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|        160|             39|2875| 1702|        0|          1|      4|          4|110|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        8|        360|            315| 344| 1702|        0|          0|      3|          2|175|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
only showing top 5 rows



In [41]:
#Criando uma varivel que ira receber essas colunas
Carros = Carros_temp.select(F.col("Consumo"), F.col("Cilindros"), F.col("Cilindradas"), F.col("HP"))

In [25]:
#Imprimindo a tabela 
Carros.show(5)

+-------+---------+-----------+---+
|Consumo|Cilindros|Cilindradas| HP|
+-------+---------+-----------+---+
|     21|        6|        160|110|
|     21|        6|        160|110|
|    228|        4|        108| 93|
|    214|        6|        258|110|
|    187|        8|        360|175|
+-------+---------+-----------+---+
only showing top 5 rows



In [42]:
#Transformando as colunas selcionada para criar uma nova coluna
vec_caracteristicas = VectorAssembler(inputCols=[("Consumo"),("Cilindros"), ("Cilindradas")], outputCol="caracteristicas")

In [43]:
#jogando para a variavel e tranformando 
Carros = vec_caracteristicas.transform(Carros)

In [48]:
#Imprimindo 
Carros.show(5)

+-------+---------+-----------+---+-----------------+
|Consumo|Cilindros|Cilindradas| HP|  caracteristicas|
+-------+---------+-----------+---+-----------------+
|     21|        6|        160|110| [21.0,6.0,160.0]|
|     21|        6|        160|110| [21.0,6.0,160.0]|
|    228|        4|        108| 93|[228.0,4.0,108.0]|
|    214|        6|        258|110|[214.0,6.0,258.0]|
|    187|        8|        360|175|[187.0,8.0,360.0]|
+-------+---------+-----------+---+-----------------+
only showing top 5 rows



In [55]:
CarrosTreino, CarrosTeste = Carros.randomSplit([0.7,0.3])

In [56]:
#Imprimindo a contagem 
print(CarrosTreino.count())
print(CarrosTeste.count())


20
12


In [60]:
reglin = LinearRegression(featuresCol="caracteristicas", labelCol="HP")
modelo = reglin.fit(CarrosTreino)